In [26]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

df = pd.read_csv("C:/Users/avillan/Documents/Varios/Personal/Teaching Assistant Position/Logistic Regression Class/Logistic Regression Data Sets/Dengue.csv")

## Converting variable to categorical data type (since that what it is)
## and then creating dummy variables
df['country'] = df['country'].astype('category')
df = pd.get_dummies(df)
df.describe()

,age,weight,malaria,died,country_India,country_Other,country_Pakistan
count,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000
mean,23.238095,129.814815,0.391534,0.312169,0.137566,0.354497,0.507937
std,5.298678,30.579380,0.489390,0.464609,0.345359,0.479631,0.501265
min,14.000000,80.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,110.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,23.000000,121.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,26.000000,140.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,45.000000,250.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
model= smf.logit(formula="died ~ age + weight + malaria + country_India + country_Other", data= df).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.567665
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   died   No. Observations:                  189
Model:                          Logit   Df Residuals:                      183
Method:                           MLE   Df Model:                            5
Date:                Sat, 26 Oct 2019   Pseudo R-squ.:                 0.08563
Time:                        11:38:00   Log-Likelihood:                -107.29
converged:                       True   LL-Null:                       -117.34
                                        LLR p-value:                  0.001200
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.3325      1.108      0.300      0.764      -1.839       2.503
age              -0.0225      0.034     -0.658      0.511      -0.089       0.044
weight           -0.0125      0.006     -1.961      0.050      -0.025   -9.66e-06
malaria           1.0544      0.380      2.775      0.006       0.310       1.799
country_India     1.2317      0.517      2.382      0.017       0.218       2.245
country_Other     0.9433      0.416      2.266      0.023       0.127       1.759
=================================================================================
"""

In [28]:
# Getting the odds ratios
model_odds = pd.DataFrame(np.exp(model.params), columns= ['OR'])
model_odds['z-value']= model.pvalues
model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())
model_odds

,OR,z-value,2.5%,97.5%
Intercept,1.394382,0.764074,0.159048,12.224607
age,0.977772,0.510649,0.914433,1.045499
weight,0.987552,0.049823,0.975269,0.999990
malaria,2.870363,0.005523,1.362952,6.044959
country_India,3.426952,0.017236,1.243677,9.442967
country_Other,2.568347,0.023440,1.135942,5.806992


In [59]:
#Comparing models using ANOVA (similar to Type 3 Analysis of Effects in SAS) 
from scipy.stats.distributions import chi2
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

model1= smf.logit(formula="died ~ age + weight + malaria + country_India + country_Other", data= df).fit()
model2= smf.logit(formula="died ~ age + weight + malaria", data= df).fit()
llf1 = model1.llf
llf2 = model2.llf


def likelihood_ratio(llmin, llmax):
    return(2*(llmax-llmin))

def lrtest(llmin, llmax):
    lr = 2 * (llmax - llmin)
    p = stats.chi2.sf(lr, 2) # llmax has 1 dof more than llmin
    return lr, p


lr, p = lrtest(llf2, llf1)
print(lr, p)

Optimization terminated successfully.
         Current function value: 0.567665
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.589628
         Iterations 6
(8.3021184414211, 0.01574772732873426)


In [75]:
df_v = pd.read_csv("C:/Users/avillan/Documents/Varios/Personal/Teaching Assistant Position/Logistic Regression Class/Logistic Regression Data Sets/Validation.csv")

df_v['country'] = df_v['country'].astype('category')
df_v = pd.get_dummies(df_v)

predictions = model.predict(df_v)
predictions

0    0.179842
1    0.301938
2    0.501247
3    0.269710
4    0.383390
dtype: float64

In [74]:
from sklearn.metrics import confusion_matrix, classification_report

predictions_binary = [ 0 if x < 0.5 else 1 for x in predictions]
print(confusion_matrix(df_v["died"], 
                       predictions_binary))

[[2 0]
 [2 1]]
